In [1]:
import pandas as pd
import sqlite3

# Extração Teste NowCasting

## Objetivos

Este notebook tem por finalidade extrair os dados de registro de óbitos do Sistema Hagape do Serviço Funerário da Prefeitura Municipal de São Paulo.

Esta extração tem por objetivo específico subsidiar a análise do grupo Observatório Covid19Br sobre necessidade de realização de técnicas de nowcasting sobre esses dados.

## Detalhamento:

A análise levará em consideração sobretudo a relação entre as seguintes variáveis, que extraímos aqui:

* dt_falecimento: a data de falecimento conforme cadastrada;
* dt_cadastro: o datetime de cadastro no sistema;
* dt_sepultamento: a data de sepultamento (para os indivíduos que foram sepultados);
* dt_cremacao: a data de cremação (para indivíduos cremados);

Além disso, geramos as seguintes colunas:

* dt_destinacao: especifica a data de destinação final do indivíduo, tenha sido ele cremado ou sepultado;
* sem_destinacao_final: 

*a tabela final contém também as colunas de "id" de todas as tabelas originais, permitindo debuggar e acrescentar dados por meio de joins posteriormente*


#### Autor:

Henrique Pougy





### Extração dos dados

A célula a seguir se conecta em um banco de dados sqlite3. Este banco de dados é gerado a partir da replicação do banco de dados original do sistema Hágape e sua posterior desnormalização.

Enquanto a duplicaçã do banco nos possibilita trabalhar com essas informações sem corrermos o risco de interferir na perfomance do sistema utilizado pelo Serviço Funerário, a sua desnormalização, valendo-se de merges específicos no banco de dados, possibilita obter mais facilmente informações estratégicas para gerenciamento da crise da pandemia de Covid-19 no município de São Paulo.



In [2]:
#abaixo realizamos a conexão no banco e geramos um cursor

con = sqlite3.connect('banco_app.db')
cursor = con.cursor()

#abaixo, listamos as tabelas do banco e guaramos seu nome na lista 'tables'
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

#por fim, printamos os nomes das tabelas
print(tables)

['dados_obito', 'cemiterios', 'contratacao', 'sepultamento', 'cremacao']


A célula abaixo extrai todos os registros das tabelas listadas acima.
Esses registros são salvos em um objeto do tipo dicionário, no qual a chave é o nome da tabela e o valor é um DataFrame contendo todos os registros.

Por fim, fechamos e o cursor e a conexão com o banco de dados, que não são mais necessários.

In [3]:
dados = {}
for table in tables:
    dados[table] = pd.read_sql(f'SELECT * FROM {table}', con) # o metodo read_sql requer a query realizada e a conexao com o banco

#abaixo fechamos as conexões com o banco de dados
cursor.close()
con.close()

### Merges

Abaixo iniciamos os merges e as filtragens de colunas, que se darão na seguinte ordem:

* cópia da tabela "dados_obito"

* left-merge com a tabela "sepultamento" 

* left merge da tabela resultante do merge anterior com a tabela "cremacao"

Tais merges são necessários para que possamos obter as datas de sepultamento - para os indivíduos que foram sepultados - ou as datas de cremacao - para os indivíduos que foram cremados


In [4]:

falec_cad = dados['dados_obito'].copy()
falec_cad_sepul = falec_cad.merge(dados['sepultamento'], on = 'id_dados_obito', how = 'left').copy()
falec_cad_sepul_crema = falec_cad_sepul.merge(dados['cremacao'], on = 'id_dados_obito', how = 'left').copy()


Em seguida, filtramos os dados apenas para as colunas que necessitamos, garantindo também seu anonimato.

É realizada uma última checagem para garantir que mantemos a mesma quantidade de registros da tabela original, de dados de sepultamento.

Os dados extraidos são assinalados à variável "df".

In [5]:
#Abaixo realizamos a filtragem das colunas para as colunas de interesse e reiniciamos o índice (boa prática para garantir que ele seja contínuo)
df = falec_cad_sepul_crema[['id_dados_obito','dt_falecimento', 'dt_cadastro', 
                         'dt_sepultamento', 'id_sepultamento','dt_cremacao', 'id_cremacao']].reset_index(drop = True).copy()

assert len(dados['dados_obito']) == len(df)

Abaixo, podemos analisar o resultado final.

Notem que a data de destinação final varia conforme o registro, podendo ser ou a data de sepultamento ou a data de cremação.

In [6]:
df.head(20)

,id_dados_obito,dt_falecimento,dt_cadastro,dt_sepultamento,id_sepultamento,dt_cremacao,id_cremacao
0,483,2019-05-14,2019-05-14 11:20:51,2019-05-14,467.0,NaN,NaN
1,484,2019-05-14,2019-05-14 16:02:57,2019-05-15,468.0,NaN,NaN
2,485,2019-05-15,2019-05-16 16:48:28,2019-05-15,469.0,NaN,NaN
3,486,2019-05-15,2019-05-17 11:55:03,2019-05-16,470.0,NaN,NaN
4,488,2019-05-16,2019-05-16 09:24:00,2019-05-16,472.0,NaN,NaN
5,489,2019-05-16,2019-05-16 09:57:16,2019-05-16,473.0,NaN,NaN
6,491,2019-04-22,2019-05-16 12:13:30,NaN,NaN,2019-03-23,144.0
7,492,2019-04-06,2019-05-17 13:04:38,2019-04-10,475.0,NaN,NaN
8,493,2019-05-16,2019-05-16 15:59:40,NaN,NaN,2019-05-16,145.0
9,494,2019-05-16,2019-05-16 22:04:43,2019-05-17,476.0,NaN,NaN


### Feature engineering/novas variáveis

Nas células a seguir, geramos as seguintes variáveis:

* dt_destinação, que indica a data de destinação final do falecido, qualquer que seja

* sem_destinacao, que indica os registros que não possuem identificação da destinação final




In [7]:

#função anônima para gerar a data de destinação - estou assinalando ela a uma variável para maior legibilidade
# essa função checa se há data de cremação para o registro, se há, ela define a data de destinação como a cremação
#caso contrário, ela define a data de sepultamento como a data de destinação

formula_dt_destinacao = lambda x: x['dt_cremacao'] if not pd.isnull(x['dt_cremacao']) else x['dt_sepultamento']

df['dt_destinacao'] = df.apply(formula_dt_destinacao, axis = 1)

Notamos, no entanto, que alguns indivíduos não possuem data de cremação ou de sepultamento.

__*Segundo conversas com o pessoal do Seviço Funerário, acredito que os indivíduos que não possuem data de destinação final sejam aqueles cujos sepultamentos/cremações ocorreram fora do município de São Paulo*__

In [8]:
df['dt_destinacao'].isnull().sum()

41

In [9]:
df[df['dt_destinacao'].isnull()].head()

,id_dados_obito,dt_falecimento,dt_cadastro,dt_sepultamento,id_sepultamento,dt_cremacao,id_cremacao,dt_destinacao
166,670,2019-06-01,2019-06-01 21:59:49,NaN,NaN,None,167.0,NaN
226,745,2019-06-02,2019-06-02 18:44:21,NaN,NaN,None,173.0,NaN
691,1276,2019-06-22,2019-06-23 07:43:44,NaN,NaN,None,219.0,NaN
759,1356,2019-06-24,2019-06-24 21:41:46,NaN,NaN,None,229.0,NaN
2752,3611,2019-08-16,2019-08-17 04:29:18,NaN,NaN,None,480.0,NaN


Por isso, geramos abaixo a variável 'sem_destinacao_final' que identifica a ausência de determinação da data de destinação final

In [10]:
df['sem_destinacao_final']  = df['dt_destinacao'].isnull()

### Exportação

Por fim, limpamos os dados, substituindo os registros nulos ("NaN" ou "None" em python) para strings vazias, facilitando a leitura.

Em seguida, salvamos o arquivo em excel

In [11]:

#substitui registros nulos
df = df.fillna('').copy().reset_index(drop = True)

#salva o arquivo em excel
df.to_excel('dados_teste_nowcasting.xlsx', index = False)